In [1]:
from google.colab import drive
drive.mount('./MyDrive')

Drive already mounted at ./MyDrive; to attempt to forcibly remount, call drive.mount("./MyDrive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [3]:
parking_df = pd.read_csv('/content/MyDrive/MyDrive/실전db.csv')
weather_df = pd.read_csv('/content/MyDrive/MyDrive/서울시_기상데이터.csv', encoding="cp949")
traffic = pd.read_excel('/content/MyDrive/MyDrive/2020교통량통합.xlsx')

In [4]:
weather_df["날짜"] = weather_df["일시"].astype(str).str[:10]
weather_df["날짜"] = pd.to_datetime(weather_df["날짜"])
weather_df['월'] = weather_df['날짜'].dt.month

df_list=[]
for month in range(1,13):
  df_list.append(weather_df[weather_df['월']==month])


# 기상청 20년 월별 데이터 평균값으로 채움
df_list[0].fillna({'강수량':60.5, '일조시간':177.9, '적설량':0.3}, inplace=True) #1
df_list[1].fillna({'강수량':53.1, '일조시간':185.0, '적설량':3.9, '지면온도':0.8, '지중온도':1.9}, inplace=True) #2
df_list[2].fillna({'강수량':16.3, '일조시간':262.9, '적설량':0.0}, inplace=True) #3
df_list[3].fillna({'강수량':16.9, '일조시간':283.3, '적설량':0.0}, inplace=True) #4
df_list[4].fillna({'강수량':112.4, '일조시간':177.9, '적설량':0.0, '지중온도':17.4}, inplace=True) #5
df_list[5].fillna({'강수량':139.6, '일조시간':206.1, '적설량':0.0, '지면온도':23.9, '지중온도':23.9}, inplace=True) #6
df_list[6].fillna({'강수량':270.4, '일조시간':119.4, '적설량':0.0, '지면온도':26.0, '지중온도':22.1}, inplace=True) #7
df_list[7].fillna({'강수량':675.7, '일조시간':74.7, '적설량':0.0, '지면온도':27.7, '지중온도':26.5, '기온':26.5}, inplace=True) #8
df_list[8].fillna({'강수량':181.5, '일조시간':174.3, '적설량':0.0}, inplace=True) #9
df_list[9].fillna({'강수량':0.0, '일조시간':227.1, '적설량':0.0}, inplace=True) #10
df_list[10].fillna({'강수량':120.1, '일조시간':171.5, '적설량':0.0}, inplace=True) #11
df_list[11].fillna({'강수량':4.6, '일조시간':203.6, '적설량':3.2}, inplace=True) #12

weather = pd.concat([df_list[0],df_list[1],df_list[2],df_list[3],df_list[4],df_list[5],df_list[6],df_list[7],df_list[8],df_list[9],df_list[10],df_list[11]])

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [5]:
daily_weather = weather.groupby("날짜").mean().drop('지점', axis=1)
print(daily_weather.shape) #2020년은 2월 29일이 있었으므로 366 row (o)

(366, 9)


In [6]:
traffic["total"] = traffic[['0시', '1시', '2시', '3시', '4시', '5시',
       '6시', '7시', '8시', '9시', '10시', '11시', '12시', '13시', '14시', '15시', '16시',
       '17시', '18시', '19시', '20시', '21시', '22시', '23시']].sum(axis=1)

daily_traffic = traffic.groupby("DATE")["DATE", 'total'].sum()

daily_traffic = daily_traffic.reset_index("DATE")
print(daily_traffic.shape)

(366, 2)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


In [7]:
daily_df = pd.merge(daily_traffic, daily_weather, left_on='DATE', right_on='날짜')
print(daily_df.shape)
daily_df.rename(columns={'기온':'Temp', '강수량':'Rain', '풍속':'Wind',
                   '습도':'Hum', '일조시간':'Daylight', '적설량':'Snow',
                   '지면온도':'GTemp', '지중온도':'UnderTemp', '월':'Month'}, inplace=True)

daily_df['Weekday'] = daily_df['DATE'].dt.weekday # 0:월 1:화 2:수 3:목 4:금 5:토 6:일

(366, 11)


In [8]:
parking_df['JOIN_DATE'] = pd.to_datetime(parking_df['JOIN_DATE'])
parking_df['DATE'] = pd.to_datetime(parking_df['DATE'])

parking_df['Weekday'] = parking_df['DATE'].dt.weekday # 0:월 1:화 2:수 3:목 4:금 5:토 6:일

In [9]:
parking_df= parking_df[parking_df['USER_ID']!=999665]

df = pd.merge(parking_df, daily_df, on = 'DATE')
print(df.shape)
df.drop('Weekday_y', axis=1, inplace=True)
df.rename(columns={'Weekday_x':'Weekday'}, inplace=True)
df

(878905, 20)


,USER_ID,JOIN_DATE,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1,Weekday,total,Temp,Rain,Wind,Hum,Daylight,Snow,GTemp,UnderTemp,Month
0,2858,2014-01-07,AA,1892,A,2020-01-01,1,GN,2,7347299.0,-2.543478,49.982609,1.152174,62.73913,92.852174,0.286957,-0.917391,2.856522,1.0
1,5647,2014-02-14,BB,182009,A,2020-01-01,1,J,2,7347299.0,-2.543478,49.982609,1.152174,62.73913,92.852174,0.286957,-0.917391,2.856522,1.0
2,33314,2014-11-20,BB,82431,A,2020-01-01,1,SC,2,7347299.0,-2.543478,49.982609,1.152174,62.73913,92.852174,0.286957,-0.917391,2.856522,1.0
3,37001,2014-12-04,BB,725,C,2020-01-01,1,MP,2,7347299.0,-2.543478,49.982609,1.152174,62.73913,92.852174,0.286957,-0.917391,2.856522,1.0
4,37819,2014-12-07,AA,220691,C,2020-01-01,1,JRR,2,7347299.0,-2.543478,49.982609,1.152174,62.73913,92.852174,0.286957,-0.917391,2.856522,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878900,1830551,2020-12-31,BB,219886,B,2020-12-31,1,GN,3,8324393.0,-12.700000,4.600000,3.400000,52.00000,203.600000,3.200000,-7.600000,2.100000,12.0
878901,1830570,2020-12-31,BB,82433,B,2020-12-31,1,CY,3,8324393.0,-12.700000,4.600000,3.400000,52.00000,203.600000,3.200000,-7.600000,2.100000,12.0
878902,1830580,2020-12-31,AA,92020,B,2020-12-31,1,JRR,3,8324393.0,-12.700000,4.600000,3.400000,52.00000,203.600000,3.200000,-7.600000,2.100000,12.0
878903,1830589,2020-12-31,BB,92437,B,2020-12-31,1,J,3,8324393.0,-12.700000,4.600000,3.400000,52.00000,203.600000,3.200000,-7.600000,2.100000,12.0


In [10]:
seoul = ['JR','J','YO','SOD','GJ','DM','JRR','SB','GB','DB','NW','EP','SD','MP','YC','GS','GR','GHN','YD','DJ','GW','SC','GN','SP','GD']

df_list=[]
for s in df['AD1']:
  if s in seoul:
    df_list.append('seoul')
  else:
    df_list.append('Nseoul')

df['Seoul'] = df_list
df = df[df['Seoul']!='Nseoul']


In [11]:
# 충성도 높은 유저 찾기
rating_list = []

for r in df['COUNT']:
  if r == 1:
    rating_list.append('Bronze')
  elif 1<r<=3:
    rating_list.append('Silver')
  elif 3<r<=5:
    rating_list.append('Gold')
  elif 5<r<=7:
    rating_list.append('Platinum')
  elif 7<r<=9:
    rating_list.append('Diamond')
  else:
    rating_list.append('VIP')

df['Rating']= rating_list

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [12]:
col=['GTemp','UnderTemp','Seoul','Wind','Daylight','JOIN_DATE','Month','Weekday']
df.drop(col, axis=1, inplace=True)

df.duplicated(keep=False)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,USER_ID,D_TYPE,STORE_ID,GOODS_TYPE,DATE,COUNT,AD1,total,Temp,Rain,Hum,Snow,Rating
0,2858,AA,1892,A,2020-01-01,1,GN,7347299.0,-2.543478,49.982609,62.73913,0.286957,Bronze
1,5647,BB,182009,A,2020-01-01,1,J,7347299.0,-2.543478,49.982609,62.73913,0.286957,Bronze
2,33314,BB,82431,A,2020-01-01,1,SC,7347299.0,-2.543478,49.982609,62.73913,0.286957,Bronze
3,37001,BB,725,C,2020-01-01,1,MP,7347299.0,-2.543478,49.982609,62.73913,0.286957,Bronze
4,37819,AA,220691,C,2020-01-01,1,JRR,7347299.0,-2.543478,49.982609,62.73913,0.286957,Bronze
...,...,...,...,...,...,...,...,...,...,...,...,...,...
878899,1830544,BB,220745,B,2020-12-31,1,SD,8324393.0,-12.700000,4.600000,52.00000,3.200000,Bronze
878900,1830551,BB,219886,B,2020-12-31,1,GN,8324393.0,-12.700000,4.600000,52.00000,3.200000,Bronze
878902,1830580,AA,92020,B,2020-12-31,1,JRR,8324393.0,-12.700000,4.600000,52.00000,3.200000,Bronze
878903,1830589,BB,92437,B,2020-12-31,1,J,8324393.0,-12.700000,4.600000,52.00000,3.200000,Bronze


In [26]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
import graphviz
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [14]:
week3_df = pd.get_dummies(df, columns=['Rating','AD1','GOODS_TYPE','D_TYPE'])
week3_df

,USER_ID,STORE_ID,DATE,COUNT,total,Temp,Rain,Hum,Snow,Rating_Bronze,Rating_Diamond,Rating_Gold,Rating_Platinum,Rating_Silver,Rating_VIP,AD1_DB,AD1_DJ,AD1_DM,AD1_EP,AD1_GB,AD1_GD,AD1_GHN,AD1_GJ,AD1_GN,AD1_GR,AD1_GS,AD1_GW,AD1_J,AD1_JR,AD1_JRR,AD1_MP,AD1_NW,AD1_SB,AD1_SC,AD1_SD,AD1_SOD,AD1_SP,AD1_YC,AD1_YD,AD1_YO,GOODS_TYPE_A,GOODS_TYPE_B,GOODS_TYPE_C,GOODS_TYPE_D,D_TYPE_AA,D_TYPE_BB,D_TYPE_CC
0,2858,1892,2020-01-01,1,7347299.0,-2.543478,49.982609,62.73913,0.286957,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,5647,182009,2020-01-01,1,7347299.0,-2.543478,49.982609,62.73913,0.286957,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,33314,82431,2020-01-01,1,7347299.0,-2.543478,49.982609,62.73913,0.286957,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0
3,37001,725,2020-01-01,1,7347299.0,-2.543478,49.982609,62.73913,0.286957,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,37819,220691,2020-01-01,1,7347299.0,-2.543478,49.982609,62.73913,0.286957,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878899,1830544,220745,2020-12-31,1,8324393.0,-12.700000,4.600000,52.00000,3.200000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
878900,1830551,219886,2020-12-31,1,8324393.0,-12.700000,4.600000,52.00000,3.200000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
878902,1830580,92020,2020-12-31,1,8324393.0,-12.700000,4.600000,52.00000,3.200000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
878903,1830589,92437,2020-12-31,1,8324393.0,-12.700000,4.600000,52.00000,3.200000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [15]:
week3_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 798421 entries, 0 to 878904
Data columns (total 47 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   USER_ID          798421 non-null  int64         
 1   STORE_ID         798421 non-null  int64         
 2   DATE             798421 non-null  datetime64[ns]
 3   COUNT            798421 non-null  int64         
 4   total            798421 non-null  float64       
 5   Temp             798421 non-null  float64       
 6   Rain             798421 non-null  float64       
 7   Hum              798421 non-null  float64       
 8   Snow             798421 non-null  float64       
 9   Rating_Bronze    798421 non-null  uint8         
 10  Rating_Diamond   798421 non-null  uint8         
 11  Rating_Gold      798421 non-null  uint8         
 12  Rating_Platinum  798421 non-null  uint8         
 13  Rating_Silver    798421 non-null  uint8         
 14  Rating_VIP       798

In [16]:
train = week3_df[week3_df['DATE'] <= '2020-09-30']
test = week3_df[week3_df['DATE'] > '2020-09-30']

In [17]:
print(train.shape, test.shape)

(566133, 47) (232288, 47)


In [18]:
# Date: timestamp ---> float64로 변경
 
train['DATE']= train['DATE'].values.astype('float64')
test['DATE']= test['DATE'].values.astype('float64')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
X_train = train.iloc[0:566133, 2:46].values
y_train = train.iloc[0:566133, [-1]].values

X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape, y_train.shape)

X_test = test.iloc[0:232288, 2:46].values
y_test = test.iloc[0:232288, [-1]].values

X_test = np.array(X_test)
y_test = np.array(y_test)

(566133, 44) (566133, 1)


In [20]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [27]:
np.random.seed(42)

gboost = GradientBoostingRegressor()
xgboost = XGBRegressor()
lightgbm = LGBMRegressor()
rdforest = RandomForestRegressor()
models = [gboost, xgboost, lightgbm, rdforest]

In [28]:
def performance(models1, x_train, y_train, x_test):
    df2 = {}

    for model in models:      
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        print("사용한 모델 :",model)
        print("MSE :",mean_squared_error(y_test, y_pred))
        print("MAE :",mean_absolute_error(y_test, y_pred))
        print()
        print()

In [29]:
performance(models, X_train, y_train, X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


사용한 모델 : GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
MSE : 4.2979250667276334e-08
MAE : 1.6379347128771223e-05


[16:51:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
사용한 모델 : XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


사용한 모델 : LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
MSE : 3.98047480587942e-13
MAE : 4.984649283450888e-08




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


사용한 모델 : RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
MSE : 0.0
MAE : 0.0




In [31]:
param_grid = {
    'loss': ['huber'],
    'n_estimators': [100],
    'learning_rate' : [0.1],
    'criterion' : ['friedman_mse'],
    'max_depth' : [30]
}

model1 = GradientBoostingRegressor()

In [33]:
# GridSearch로 가장 좋은 파라미터 찾기
GCV = GridSearchCV(model1, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=5)

# 모델 fitting
GCV.fit(X_train, y_train)

# 가장 좋은 파라미터 찾기
print("Best Param :", GCV.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   41.6s remaining:  1.0min
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   41.7s finished
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Param : {'criterion': 'friedman_mse', 'learning_rate': 0.1, 'loss': 'huber', 'max_depth': 30, 'n_estimators': 100}


In [35]:
# 가장 좋은 파라미터로 모델 설정
model1=GCV.best_estimator_ 

# 예측
y_pred1 = model1.predict(X_test)

# MSE, MAE값
print("MSE :",mean_squared_error(y_test, y_pred1))
print("MAE :",mean_absolute_error(y_test, y_pred1))

MSE : 0.0005639550902328145
MAE : 0.0005639550902328145
